# STEM Cell Population Wright-Fisher Algorithm Fixation Analysis

In this example notebook we present an example of how to forward simulate a population of STEM cells in the context of a fixed size tumor. We assume a constant number of cells in the population at all times -- only the counts of the different species of cells change in time:

- wild type (WT)
- with cell intrinsic mutations that increase fitness (A)
- with mutations that give evolutionary advantage based on environmental factors such as level of cytokines (B).

Fot the purposes of this notebook we consider the environmental conditions such that the B cells always have a selective advantage over their wilde type counterpart.

In [1]:
# Load necessary libraries
import os
import numpy as np
import pandas as pd
from scipy.stats import gamma
import cmmlinflam as ci
import matplotlib
import plotly.graph_objects as go
from matplotlib import pyplot as plt

## Plot output of Wright-Fisher for the different species of cells

In [2]:
from plotly.subplots import make_subplots

colours = ['blue', 'red', 'green', 'purple', 'orange', 'black', 'gray', 'pink']
species = ['WT', 'A', 'B']

## Compute mean time to fixation and probability of fixation of A cells when no B are present

## No selective advatange + No mutation

### Start from 1/2 and 1/2 WT ws A

In [3]:
# Set initial population state WT - A - B
initial_population = [50, 50, 0]

# Set baseline growth rate
alpha = 0.5

# Set selective advantages for mutated cells
s = 0
r = 0

# Set mutation rates
mu_A = 0
mu_B = 0

# Coalesce into paramater vector
parameters = initial_population
parameters.extend([alpha, s, r, mu_A, mu_B])

In [4]:
# Instantiate algorithm
algorithm = ci.StemWF()

# Select number of simulations
num_simulations = 100000

computation_time = np.empty(num_simulations, dtype=np.int)
fixed_state = np.empty(num_simulations, dtype=np.str)

for sim in range(num_simulations):
    computation_time[sim], fixed_state[sim] = algorithm.simulate_fixation(parameters)

mean_computation_time = np.mean(computation_time)
prob_fix = (fixed_state == 'A').sum()/num_simulations

mean_comp_time_fix_A = np.mean(computation_time[(fixed_state == 'A')])

print('Average time to fixation: ', mean_computation_time)
print('Probability of fixation of cell type A: ', prob_fix)
print('Average time to illness when fixation at A: ', mean_comp_time_fix_A)

Average time to fixation:  136.86899
Probability of fixation of cell type A:  0.50281
Average time to illness when fixation at A:  137.032020047334


### Start from 1 A vs all other WT

In [5]:
# Set initial population state WT - A - B
initial_population = [99, 1, 0]

# Set baseline growth rate
alpha = 0.5

# Set selective advantages for mutated cells
s = 0
r = 0

# Set mutation rates
mu_A = 0
mu_B = 0

# Coalesce into paramater vector
parameters = initial_population
parameters.extend([alpha, s, r, mu_A, mu_B])

In [6]:
# Instantiate algorithm
algorithm = ci.StemWF()

# Select number of simulations
num_simulations = 1000

computation_time = np.empty(num_simulations, dtype=np.int)
fixed_state = np.empty(num_simulations, dtype=np.str)

for sim in range(num_simulations):
    computation_time[sim], fixed_state[sim] = algorithm.simulate_fixation(parameters)

mean_computation_time = np.mean(computation_time)
prob_fix = (fixed_state == 'A').sum()/num_simulations

mean_comp_time_fix_A = np.mean(computation_time[(fixed_state == 'A')])

print('Average time to fixation: ', mean_computation_time)
print('Probability of fixation of cell type A: ', prob_fix)
print('Average time to illness when fixation at A: ', mean_comp_time_fix_A)

Average time to fixation:  10.125
Probability of fixation of cell type A:  0.011
Average time to illness when fixation at A:  179.27272727272728


### Plot transition probability curves

In [7]:
# Plot transition probabilities
sep_algo = ci.StemWF()

sep_algo.N = int(np.sum(np.asarray(initial_population)))

sep_algo.alpha_A = alpha + s
sep_algo.alpha_B = alpha + r
sep_algo.alpha_WT = alpha

sep_algo.mu_A = mu_A
sep_algo.mu_B = mu_B

# Assuming no Bs in the population
trans_prob = np.empty((sep_algo.N+1, 2))

for i in range(sep_algo.N+1):
    trans_prob[i, 0] = sep_algo._prob_A_to_WT(i, sep_algo.N - i, 0)
    trans_prob[i, 1] = sep_algo._prob_WT_to_A(i, sep_algo.N - i, 0)

In [8]:
# Trace names - represent the transition probabilities used for the simulation
trace_name = ['A->WT', 'WT->A']

fig = go.Figure()

# Add traces of the transition probabilities
for c in range(trans_prob.shape[1]):
    fig.add_trace(
        go.Scatter(
            y=trans_prob[:, c],
            x=list(range(sep_algo.N+1)),
            mode='lines',
            name=trace_name[c],
            line_color=colours[c]
        )
    )

fig.update_layout(
    title='Transition probabilities for edge case with no B cells - No selective advantage + No mutation',
    width=1000, 
    height=600,
    plot_bgcolor='white',
    xaxis=dict(linecolor='black'),
    yaxis=dict(linecolor='black'),
    )

## selective advatange + No mutation

### Start from 1/2 and 1/2 WT ws A

In [9]:
# Set initial population state WT - A - B
initial_population = [50, 50, 0]

# Set baseline growth rate
alpha = 1

# Set selective advantages for mutated cells
s = 0.1
r = 0

# Set mutation rates
mu_A = 0
mu_B = 0

# Coalesce into paramater vector
parameters = initial_population
parameters.extend([alpha, s, r, mu_A, mu_B])

In [10]:
# Instantiate algorithm
algorithm = ci.StemWF()

# Select number of simulations
num_simulations = 100000

computation_time = np.empty(num_simulations, dtype=np.int)
fixed_state = np.empty(num_simulations, dtype=np.str)

for sim in range(num_simulations):
    computation_time[sim], fixed_state[sim] = algorithm.simulate_fixation(parameters)

mean_computation_time = np.mean(computation_time)
prob_fix = (fixed_state == 'A').sum()/num_simulations

mean_comp_time_fix_A = np.mean(computation_time[(fixed_state == 'A')])

print('Average time to fixation: ', mean_computation_time)
print('Probability of fixation of cell type A: ', prob_fix)
print('Average time to illness when fixation at A: ', mean_comp_time_fix_A)

Average time to fixation:  37.37094
Probability of fixation of cell type A:  0.99994
Average time to illness when fixation at A:  37.37027221633298


### Start from 1 A vs all other WT

In [11]:
# Set initial population state WT - A - B
initial_population = [99, 1, 0]

# Set baseline growth rate
alpha = 1

# Set selective advantages for mutated cells
s = 0.1
r = 0

# Set mutation rates
mu_A = 0
mu_B = 0

# Coalesce into paramater vector
parameters = initial_population
parameters.extend([alpha, s, r, mu_A, mu_B])

In [12]:
# Instantiate algorithm
algorithm = ci.StemWF()

# Select number of simulations
num_simulations = 100000

computation_time = np.empty(num_simulations, dtype=np.int)
fixed_state = np.empty(num_simulations, dtype=np.str)

for sim in range(num_simulations):
    computation_time[sim], fixed_state[sim] = algorithm.simulate_fixation(parameters)

mean_computation_time = np.mean(computation_time)
prob_fix = (fixed_state == 'A').sum()/num_simulations

mean_comp_time_fix_A = np.mean(computation_time[(fixed_state == 'A')])

print('Average time to fixation: ', mean_computation_time)
print('Probability of fixation of cell type A: ', prob_fix)
print('Average time to illness when fixation at A: ', mean_comp_time_fix_A)

Average time to fixation:  15.8566
Probability of fixation of cell type A:  0.17545
Average time to illness when fixation at A:  70.41476204046737


### Plot transition probability curves

In [13]:
# Plot transition probabilities
sep_algo = ci.StemWF()

sep_algo.N = int(np.sum(np.asarray(initial_population)))

sep_algo.alpha_A = alpha + s
sep_algo.alpha_B = alpha + r
sep_algo.alpha_WT = alpha

sep_algo.mu_A = mu_A
sep_algo.mu_B = mu_B

# Assuming no Bs in the population
trans_prob = np.empty((sep_algo.N+1, 2))

for i in range(sep_algo.N+1):
    trans_prob[i, 0] = sep_algo._prob_A_to_WT(i, sep_algo.N - i, 0)
    trans_prob[i, 1] = sep_algo._prob_WT_to_A(i, sep_algo.N - i, 0)

In [14]:
# Trace names - represent the transition probabilities used for the simulation
trace_name = ['A->WT', 'WT->A']

fig = go.Figure()

# Add traces of the transition probabilities
for c in range(trans_prob.shape[1]):
    fig.add_trace(
        go.Scatter(
            y=trans_prob[:, c],
            x=list(range(sep_algo.N+1)),
            mode='lines',
            name=trace_name[c],
            line_color=colours[c]
        )
    )

fig.update_layout(
    title='Transition probabilities for edge case with no B cells - With selective advantage + No mutation',
    width=1000, 
    height=600,
    plot_bgcolor='white',
    xaxis=dict(linecolor='black'),
    yaxis=dict(linecolor='black'),
    )

## No selective advatange + Mutation

### Start from 1/2 and 1/2 WT ws A

In [15]:
# Set initial population state WT - A - B
initial_population = [50, 50, 0]

# Set baseline growth rate
alpha = 1

# Set selective advantages for mutated cells
s = 0
r = 0

# Set mutation rates
mu_A = 0.001
mu_B = 0

# Coalesce into paramater vector
parameters = initial_population
parameters.extend([alpha, s, r, mu_A, mu_B])

In [16]:
# Instantiate algorithm
algorithm = ci.StemWF()

# Select number of simulations
num_simulations = 100000

computation_time = np.empty(num_simulations, dtype=np.int)
fixed_state = np.empty(num_simulations, dtype=np.str)

for sim in range(num_simulations):
    computation_time[sim], fixed_state[sim] = algorithm.simulate_fixation(parameters)

mean_computation_time = np.mean(computation_time)
prob_fix = (fixed_state == 'A').sum()/num_simulations

mean_comp_time_fix_A = np.mean(computation_time[(fixed_state == 'A')])

print('Average time to fixation: ', mean_computation_time)
print('Probability of fixation of cell type A: ', prob_fix)
print('Average time to illness when fixation at A: ', mean_comp_time_fix_A)

### Start from 1 A vs all other WT

In [ ]:
# Set initial population state WT - A - B
initial_population = [99, 1, 0]

# Set baseline growth rate
alpha = 1

# Set selective advantages for mutated cells
s = 0
r = 0

# Set mutation rates
mu_A = 0.001
mu_B = 0

# Coalesce into paramater vector
parameters = initial_population
parameters.extend([alpha, s, r, mu_A, mu_B])

In [ ]:
# Instantiate algorithm
algorithm = ci.StemWF()

# Select number of simulations
num_simulations = 100000

computation_time = np.empty(num_simulations, dtype=np.int)
fixed_state = np.empty(num_simulations, dtype=np.str)

for sim in range(num_simulations):
    computation_time[sim], fixed_state[sim] = algorithm.simulate_fixation(parameters)

mean_computation_time = np.mean(computation_time)
prob_fix = (fixed_state == 'A').sum()/num_simulations

mean_comp_time_fix_A = np.mean(computation_time[(fixed_state == 'A')])

print('Average time to fixation: ', mean_computation_time)
print('Probability of fixation of cell type A: ', prob_fix)
print('Average time to illness when fixation at A: ', mean_comp_time_fix_A)

Average time to fixation:  1657.834
Probability of fixation of cell type A:  0.02223
Average time to illness when fixation at A:  20838.731443994602


### Plot transition probability curves

In [ ]:
# Plot transition probabilities
sep_algo = ci.StemWF()

sep_algo.N = int(np.sum(np.asarray(initial_population)))

sep_algo.alpha_A = alpha + s
sep_algo.alpha_B = alpha + r
sep_algo.alpha_WT = alpha

sep_algo.mu_A = 0.1
sep_algo.mu_B = mu_B

# Assuming no Bs in the population
trans_prob = np.empty((sep_algo.N+1, 2))

for i in range(sep_algo.N+1):
    trans_prob[i, 0] = sep_algo._prob_A_to_WT(i, sep_algo.N - i, 0)
    trans_prob[i, 1] = sep_algo._prob_WT_to_A(i, sep_algo.N - i, 0)

In [ ]:
# Trace names - represent the transition probabilities used for the simulation
trace_name = ['A->WT', 'WT->A']

fig = go.Figure()

# Add traces of the transition probabilities
for c in range(trans_prob.shape[1]):
    fig.add_trace(
        go.Scatter(
            y=trans_prob[:, c],
            x=list(range(sep_algo.N+1)),
            mode='lines',
            name=trace_name[c],
            line_color=colours[c]
        )
    )

fig.update_layout(
    title='Transition probabilities for edge case with no B cells - No selective advantage + With mutation',
    width=1000, 
    height=600,
    plot_bgcolor='white',
    xaxis=dict(linecolor='black'),
    yaxis=dict(linecolor='black'),
    )

## Compute mean time to fixation and probability of fixation of B cells when no A are present

## No selective advatange + No mutation

### Start from 1/2 and 1/2 WT ws B

In [ ]:
# Set initial population state WT - A - B
initial_population = [50, 0, 50]

# Set baseline growth rate
alpha = 0.5

# Set selective advantages for mutated cells
s = 0
r = 0
 
# Set mutation rates
mu_A = 0
mu_B = 0

# Coalesce into paramater vector
parameters = initial_population
parameters.extend([alpha, s, r, mu_A, mu_B])

# Choose switching times
switch_times = np.stack((np.array(range(200)) * 500, np.array(range(1, 201)) % 2), axis=1).tolist()
#switch_times = np.stack((np.array(range(200)) * 500, np.ones(200, dtype=np.int)), axis=1).tolist()

In [ ]:
# Instantiate algorithm
algorithm = ci.StemWFTIMEVAR()

# Select number of simulations
num_simulations = 100000

computation_time = np.empty(num_simulations, dtype=np.int)
fixed_state = np.empty(num_simulations, dtype=np.str)

for sim in range(num_simulations):
    computation_time[sim], fixed_state[sim] = algorithm.simulate_fixation(parameters, switch_times)

mean_computation_time = np.mean(computation_time)
prob_fix = (fixed_state == 'B').sum()/num_simulations

mean_comp_time_fix_B = np.mean(computation_time[(fixed_state == 'B')])

print('Average time to fixation: ', mean_computation_time)
print('Probability of fixation of cell type B: ', prob_fix)
print('Average time to illness when fixation at B: ', mean_comp_time_fix_B)

Average time to fixation:  13646.78
Probability of fixation of cell type B:  0.49896
Average time to illness when fixation at B:  13649.799583132917


### Start from 1 B vs all other WT

In [ ]:
# Set initial population state WT - A - B
initial_population = [99, 0, 1]

# Set baseline growth rate
alpha = 0.5

# Set selective advantages for mutated cells
s = 0
r = 0
 
# Set mutation rates
mu_A = 0
mu_B = 0

# Coalesce into paramater vector
parameters = initial_population
parameters.extend([alpha, s, r, mu_A, mu_B])

# Choose switching times
switch_times = np.stack((np.array(range(200)) * 500, np.array(range(1, 201)) % 2), axis=1).tolist()
#switch_times = np.stack((np.array(range(200)) * 500, np.ones(200, dtype=np.int)), axis=1).tolist()

In [ ]:
# Instantiate algorithm
algorithm = ci.StemWFTIMEVAR()

# Select number of simulations
num_simulations = 100000

computation_time = np.empty(num_simulations, dtype=np.int)
fixed_state = np.empty(num_simulations, dtype=np.str)

for sim in range(num_simulations):
    computation_time[sim], fixed_state[sim] = algorithm.simulate_fixation(parameters, switch_times)

mean_computation_time = np.mean(computation_time)
prob_fix = (fixed_state == 'B').sum()/num_simulations

mean_comp_time_fix_B = np.mean(computation_time[(fixed_state == 'B')])

print('Average time to fixation: ', mean_computation_time)
print('Probability of fixation of cell type B: ', prob_fix)
print('Average time to illness when fixation at B: ', mean_comp_time_fix_B)

Average time to fixation:  1064.207
Probability of fixation of cell type B:  0.01011
Average time to illness when fixation at B:  20030.86053412463


### Plot transition probability curves

In [ ]:
# Plot transition probabilities
sep_algo = ci.StemWFTIMEVAR()

sep_algo.N = int(np.sum(np.asarray(initial_population)))

sep_algo.alpha_A = alpha + s
sep_algo.alpha_B = alpha + r
sep_algo.alpha_WT = alpha
sep_algo.switches =  np.asarray(switch_times)

sep_algo.mu_A = mu_B
sep_algo.mu_B = mu_B

# Assuming no Bs in the population
trans_prob = np.empty((sep_algo.N+1, 4))

for i in range(sep_algo.N+1):
    trans_prob[i, 0] = sep_algo._prob_B_to_WT(1, i, 0,sep_algo.N - i)
    trans_prob[i, 1] = sep_algo._prob_WT_to_B(1, i, 0, sep_algo.N - i)
    trans_prob[i, 2] = sep_algo._prob_B_to_WT(501, i, 0, sep_algo.N - i)
    trans_prob[i, 3] = sep_algo._prob_WT_to_B(501, i, 0, sep_algo.N - i)

In [ ]:
# Trace names - represent the transition probabilities used for the simulation
trace_name = ['B->WT env ON', 'WT->B env ON', 'B->WT env OFF', 'WT->B env OFF']

fig = go.Figure()

# Add traces of the transition probabilities
for c in range(trans_prob.shape[1]):
    fig.add_trace(
        go.Scatter(
            y=trans_prob[:, c],
            x=list(range(sep_algo.N+1)),
            mode='lines',
            name=trace_name[c],
            line_color=colours[c]
        )
    )

fig.update_layout(
    title='Transition probabilities for edge case with no B cells - No selective advantage + With mutation',
    width=1000, 
    height=600,
    plot_bgcolor='white',
    xaxis=dict(linecolor='black'),
    yaxis=dict(linecolor='black'),
    )

## selective advatange + No mutation

### Start from 1/2 and 1/2 WT ws B

In [ ]:
# Set initial population state WT - A - B
initial_population = [50, 0, 50]

# Set baseline growth rate
alpha = 1

# Set selective advantages for mutated cells
s = 0
r = 0.1
 
# Set mutation rates
mu_A = 0
mu_B = 0

# Coalesce into paramater vector
parameters = initial_population
parameters.extend([alpha, s, r, mu_A, mu_B])

# Choose switching times
switch_times = np.stack((np.array(range(200)) * 500, np.array(range(1, 201)) % 2), axis=1).tolist()
#switch_times = np.stack((np.array(range(200)) * 500, np.ones(200, dtype=np.int)), axis=1).tolist()

In [ ]:
# Instantiate algorithm
algorithm = ci.StemWFTIMEVAR()

# Select number of simulations
num_simulations = 100000

computation_time = np.empty(num_simulations, dtype=np.int)
fixed_state = np.empty(num_simulations, dtype=np.str)

for sim in range(num_simulations):
    computation_time[sim], fixed_state[sim] = algorithm.simulate_fixation(parameters, switch_times)

mean_computation_time = np.mean(computation_time)
prob_fix = (fixed_state == 'B').sum()/num_simulations

mean_comp_time_fix_B = np.mean(computation_time[(fixed_state == 'B')])

print('Average time to fixation: ', mean_computation_time)
print('Probability of fixation of cell type B: ', prob_fix)
print('Average time to illness when fixation at B: ', mean_comp_time_fix_B)

Average time to fixation:  3739.284
Probability of fixation of cell type B:  0.99988
Average time to illness when fixation at B:  3739.331719806377


### Start from 1 B vs all other WT

In [ ]:
# Set initial population state WT - A - B
initial_population = [99, 0, 1]

# Set baseline growth rate
alpha = 1

# Set selective advantages for mutated cells
s = 0
r = 0.1
 
# Set mutation rates
mu_A = 0
mu_B = 0

# Coalesce into paramater vector
parameters = initial_population
parameters.extend([alpha, s, r, mu_A, mu_B])

# Choose switching times
switch_times = np.stack((np.array(range(200)) * 500, np.array(range(1, 201)) % 2), axis=1).tolist()
#switch_times = np.stack((np.array(range(200)) * 500, np.ones(200, dtype=np.int)), axis=1).tolist()

In [ ]:
# Instantiate algorithm
algorithm = ci.StemWFTIMEVAR()

# Select number of simulations
num_simulations = 100000

computation_time = np.empty(num_simulations, dtype=np.int)
fixed_state = np.empty(num_simulations, dtype=np.str)

for sim in range(num_simulations):
    computation_time[sim], fixed_state[sim] = algorithm.simulate_fixation(parameters, switch_times)

mean_computation_time = np.mean(computation_time)
prob_fix = (fixed_state == 'B').sum()/num_simulations

mean_comp_time_fix_B = np.mean(computation_time[(fixed_state == 'B')])

print('Average time to fixation: ', mean_computation_time)
print('Probability of fixation of cell type B: ', prob_fix)
print('Average time to illness when fixation at B: ', mean_comp_time_fix_B)

Average time to fixation:  1592.619
Probability of fixation of cell type B:  0.17656
Average time to illness when fixation at B:  7048.210240144993


### Plot transition probability curves

In [ ]:
# Plot transition probabilities
sep_algo = ci.StemWFTIMEVAR()

sep_algo.N = int(np.sum(np.asarray(initial_population)))

sep_algo.alpha_A = alpha + s
sep_algo.alpha_B = alpha + r
sep_algo.alpha_WT = alpha
sep_algo.switches =  np.asarray(switch_times)

sep_algo.mu_A = mu_B
sep_algo.mu_B = mu_B

# Assuming no Bs in the population
trans_prob = np.empty((sep_algo.N+1, 4))

for i in range(sep_algo.N+1):
    trans_prob[i, 0] = sep_algo._prob_B_to_WT(1, i, 0,sep_algo.N - i)
    trans_prob[i, 1] = sep_algo._prob_WT_to_B(1, i, 0, sep_algo.N - i)
    trans_prob[i, 2] = sep_algo._prob_B_to_WT(501, i, 0, sep_algo.N - i)
    trans_prob[i, 3] = sep_algo._prob_WT_to_B(501, i, 0, sep_algo.N - i)

In [ ]:
# Trace names - represent the transition probabilities used for the simulation
trace_name = ['B->WT env ON', 'WT->B env ON', 'B->WT env OFF', 'WT->B env OFF']

fig = go.Figure()

# Add traces of the transition probabilities
for c in range(trans_prob.shape[1]):
    fig.add_trace(
        go.Scatter(
            y=trans_prob[:, c],
            x=list(range(sep_algo.N+1)),
            mode='lines',
            name=trace_name[c],
            line_color=colours[c]
        )
    )

fig.update_layout(
    title='Transition probabilities for edge case with no B cells - No selective advantage + With mutation',
    width=1000, 
    height=600,
    plot_bgcolor='white',
    xaxis=dict(linecolor='black'),
    yaxis=dict(linecolor='black'),
    )